## Imports

In [1]:
import pandas as pd

## Leitura da base

### Primeira leitura para checagem de dados

In [2]:
df_carts = pd.read_parquet('C:/Users/candi/OneDrive/Documentos/GitHub/case_enjoei/datasets/bronze/carts.parquet')

In [3]:
df_carts

,id,userId,date,products,__v
0,1,1,2020-03-02T00:00:00.000Z,"[{'productId': 1, 'quantity': 4}, {'productId'...",0
1,2,1,2020-01-02T00:00:00.000Z,"[{'productId': 2, 'quantity': 4}, {'productId'...",0
2,3,2,2020-03-01T00:00:00.000Z,"[{'productId': 1, 'quantity': 2}, {'productId'...",0
3,4,3,2020-01-01T00:00:00.000Z,"[{'productId': 1, 'quantity': 4}]",0
4,5,3,2020-03-01T00:00:00.000Z,"[{'productId': 7, 'quantity': 1}, {'productId'...",0
5,6,4,2020-03-01T00:00:00.000Z,"[{'productId': 10, 'quantity': 2}, {'productId...",0
6,7,8,2020-03-01T00:00:00.000Z,"[{'productId': 18, 'quantity': 1}]",0


# Tratamentos da base

In [4]:
# O método explode() transforma cada elemento de uma lista em uma linha separada.
df_exploded = df_carts.explode('products')

In [5]:
df_exploded

,id,userId,date,products,__v
0,1,1,2020-03-02T00:00:00.000Z,"{'productId': 1, 'quantity': 4}",0
0,1,1,2020-03-02T00:00:00.000Z,"{'productId': 2, 'quantity': 1}",0
0,1,1,2020-03-02T00:00:00.000Z,"{'productId': 3, 'quantity': 6}",0
1,2,1,2020-01-02T00:00:00.000Z,"{'productId': 2, 'quantity': 4}",0
1,2,1,2020-01-02T00:00:00.000Z,"{'productId': 1, 'quantity': 10}",0
1,2,1,2020-01-02T00:00:00.000Z,"{'productId': 5, 'quantity': 2}",0
2,3,2,2020-03-01T00:00:00.000Z,"{'productId': 1, 'quantity': 2}",0
2,3,2,2020-03-01T00:00:00.000Z,"{'productId': 9, 'quantity': 1}",0
3,4,3,2020-01-01T00:00:00.000Z,"{'productId': 1, 'quantity': 4}",0
4,5,3,2020-03-01T00:00:00.000Z,"{'productId': 7, 'quantity': 1}",0


In [6]:
# extração  productId e quantity de cada dicionário

'''São Criadas  novas colunas chamadas 'productId' e 'quantity' O apply() executa a função lambda para cada linha.
A função lambda lambda x: x['productId'] pega cada dicionário (x) e extrai o valor da chave 'productId' gerando uma nova coluna.
O mesmo se repete para 'quantity' '''

df_exploded['product_id'] = df_exploded['products'].apply(lambda x: x['productId'])
df_exploded['quantity'] = df_exploded['products'].apply(lambda x: x['quantity'])

### Remoção de colunas

In [7]:
# Remoção da coluna original de "products" e a "__v" por não haver uso para a mesma.
df_exploded = df_exploded.drop('products', axis=1)
df_exploded = df_exploded.drop('__v', axis=1)

### Conferência dos dados

In [8]:
df_exploded

,id,userId,date,product_id,quantity
0,1,1,2020-03-02T00:00:00.000Z,1,4
0,1,1,2020-03-02T00:00:00.000Z,2,1
0,1,1,2020-03-02T00:00:00.000Z,3,6
1,2,1,2020-01-02T00:00:00.000Z,2,4
1,2,1,2020-01-02T00:00:00.000Z,1,10
1,2,1,2020-01-02T00:00:00.000Z,5,2
2,3,2,2020-03-01T00:00:00.000Z,1,2
2,3,2,2020-03-01T00:00:00.000Z,9,1
3,4,3,2020-01-01T00:00:00.000Z,1,4
4,5,3,2020-03-01T00:00:00.000Z,7,1


## Alteração e padronização nos nomes das colunas

In [9]:
# Usando rename com inplace=True (modifica o DataFrame original)
df_exploded.rename(columns={'id': 'carts_id', 'userId': 'user_id'}, inplace=True)

## Tratamento de datas

In [10]:
# Converção da coluna para datetime e extração da data
df_exploded['date'] = pd.to_datetime(df_exploded['date']).dt.date

In [11]:
df_exploded

,carts_id,user_id,date,product_id,quantity
0,1,1,2020-03-02,1,4
0,1,1,2020-03-02,2,1
0,1,1,2020-03-02,3,6
1,2,1,2020-01-02,2,4
1,2,1,2020-01-02,1,10
1,2,1,2020-01-02,5,2
2,3,2,2020-03-01,1,2
2,3,2,2020-03-01,9,1
3,4,3,2020-01-01,1,4
4,5,3,2020-03-01,7,1


## Conferência dos dados nulos

In [12]:
# Soma de todos os valores nulos por coluna
df_exploded.isnull().sum()

carts_id      0
user_id       0
date          0
product_id    0
quantity      0
dtype: int64

No caso dessa base, não houve a presença de dados nulos

## Salvando na camada prata

In [13]:
df_exploded.to_parquet('C:/Users/candi/OneDrive/Documentos/GitHub/case_enjoei/datasets/silver/carts.parquet', index=False)